In [1]:
import sys
import os
import json
import time
import pymongo
import urllib
import pandas as pd
import numpy as np
import urllib
import urllib.request

# sys.path.append(os.environ.get("PYTHONPATH"))

try:
    import createNewData.data.config as in_config
    from createNewData.pypackages.Azure import Azure
    from createNewData.pypackages.urlHandler import UrlHandler
    AzurePackage = Azure(in_config)
    Url = UrlHandler(in_config)
    
    
except ImportError as e:
    print("Failed to import critical modules for this script.")
    print("Please confirm that files exist in the correct locations.")
    print(e)

# Upload to SQL Database
The below can upload an entire table from a txt or csv file to the Azure SQL Database.
Required fields:
* "UploadToSQL" is the class name for uploading tables to the database.
* tablepath is the path the file is located in.
* tablename is the name of the new table to be created.

In [ ]:
df = pd.read_csv(in_config.shapes)

SqlDataCursor = AzurePackage("UploadToSQL",
                              df,
                              "shapes")

# Return All Unique Shape Id's From SQL Database
The below collects all unique values in a specific column of a database.
Required fields:
* "SelectDistinct" is the class name for collecting the unique values.
* shape_id is the column name.
* [[dbo].[shapes]] is the name of the table to be queried.

In [2]:
shapeIds = AzurePackage("SelectDistinct",
                              "shape_id",
                              "[dbo].[shapes]")

InterfaceError: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')

# Collect Elevation Data and Upload to MongoDB
The below collects the longtitude and lattitude for elevation data from the shapes table.
It then generates and send the request to the open elevations URL.
Finally it stores this data to the mongoDB database in Azure Cosmos.

In [ ]:
shapeList = []
AzurePackage("dropMongoColl","shapes")
for each in shapeIds.iterrows():
    try:
        # Generate the Pandas table of all the Longtitudes and Latitudes
        # for each shape
        elevations = AzurePackage("SelectLongLat",
                                "[shape_id],[shape_pt_lat],[shape_pt_lon]",
                                "[dbo].[shapes]",
                                "[shape_id]",
                                each[1][0])

        # Generate the Json document for upload to MongoDB
        shapeData = Url("mineElevationData",elevations)
    except urllib.error.HTTPError as e:
        print(in_config.URLOOD)
        print(e)

    try:
        # Upload the Json document to MongoDB
        if not len(shapeData) == 0:
            AzurePackage("UploadToMongo","shapes",shapeData)
        else:
            raise Exception(in_config.NDIDF)
    except TypeError as e:
        print(in_config.TEC)
    except pymongo.errors.DuplicateKeyError as e:
        print(in_config.FIDB)
    except Exception as e:
        print(in_config.UNKMGO)
        print(e)



# Read Data From MongoDB and Write to Pandas DataFrame
This reads data from the raw Json files in the MongoDB database and imports them to a dataframe that contains only unique values. 
Removing any rows where any full row duplicates exist.

In [ ]:
listOfObjects = []
listOfElevations = []

dbcollections = AzurePackage("SelectFromMongo")

for each in dbcollections.find():
    for key, value in each.items():
        if type(value) is list:
            listOfObjects.append(value)
    

for each in listOfObjects:
    for elevation in each:
        listOfElevations.append(elevation)
df = pd.DataFrame(listOfElevations)
dfTrimmed = df.drop_duplicates()
print(dfTrimmed)

# Upload Trimmed Elevations DataFrame to SQL

need to add functionality to replace old data with new data if required.


In [ ]:
df = pd.read_csv(in_config.shapes)
SqlDataCursor = AzurePackage("UploadToSQL",
                              dfTrimmed,
                              "elevations")

# Connect the shape and elevation schema together by joining their longtitude and latatude values
This will either be a method in the Rshiny app to collect or we simply create a new database from this data but this seems a bit verbose.

In [3]:
conn = AzurePackage("AzureDBConn", in_config.connQuote)
SQLString = in_config.SQLElevation
df = pd.read_sql(SQLString, conn)
conn.close()

OperationalError: ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]Named Pipes Provider: Could not open a connection to SQL Server [53].  (53) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Invalid connection string attribute (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (53)')

# Test for missing Elevations
This tests that all the elevation data was collected correctly by summing the values of the elevations and therefore ruling out any NaN values.
An exception will be raised here in the final script to indicate that the elevation collection was unsuccessfull.

In [ ]:
sumElevation = df["elevation"].sum())
if type(sumElevation) == int:
    print("Elevations collected correctly")
else: 
    print("Failed to collect all elevations, please try again.")

# Collect real time data and upload to Mongo
This collects the real time data as a json file and overwrites the collection in the mongoDB database.
* URL used https://gtfsr.transportforireland.ie


In [4]:
url = in_config.url2
headers = in_config.RTIheaders
response = Url("callURL", url, {}, headers)
JsonData = response.read().decode('utf8').replace("'", '"')
RTIgtfs = json.loads(JsonData)
AzurePackage("dropMongoColl","RTIgtfs")
try:
    AzurePackage("UploadToMongo","RTIgtfs",RTIgtfs)
except pymongo.errors.WriteError as e:
    print("File is too large to save to CosmosDB.")
    print(type(e))


232


ServerSelectionTimeoutError: jamescookland.mongo.cosmos.azure.com:10255: timed out, Timeout: 30s, Topology Description: <TopologyDescription id: 6077fb233603b35251404d80, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('jamescookland.mongo.cosmos.azure.com', 10255) server_type: Unknown, rtt: None, error=NetworkTimeout('jamescookland.mongo.cosmos.azure.com:10255: timed out')>]>

In [7]:
RTIgtfs.find({'id': '1510148.5.10-203-e20-1.557.I'})

AttributeError: 'dict' object has no attribute 'find'

# Connect to team database

In [3]:
conn = AzurePackage("AzureDBConn", in_config.teamConnQuote)
cursor = conn.cursor()
a = cursor.execute("select DB_NAME(db_id())")
print(a.fetchall())
conn.close()

[('electricbus-eastus-prod', )]
